In [1]:
import pandas as pd

In [2]:
path='/Users/juanignaciosolerno/Downloads/julius/datasets/title_discovery_no_parsed_jt (3).csv'
df = pd.read_csv(path)
df.sample(30)


,doc_id,doc_title,doc_company,doc_location,parsed_jt,source,Job Title Present
5124,98661d265166270b,Cust Svc Rep I (Temporary),Puget Sound Energy,"Bothell, WA 98011",NaN,indeed,Yes
10162,7415221,Deliver with Uber Eats,Uber,"West Reading, PA",NaN,linkedin,No
9329,2756348,Deliver with Uber Eats,Uber,"Mardela Springs, MD",NaN,linkedin,No
5989,6402219,Deliver with Uber Eats,Uber,"Lake Mary, FL",NaN,linkedin,No
4442,2264af15d0baea7d,Bulverde Produce - Perishables Rep - Part-Time,HEB,"Spring Branch, TX 78070",NaN,indeed,Yes
14005,1900899,Deliver with Uber Eats,Uber,"Wolfe City, TX",NaN,linkedin,No
5737,1604743,Deliver with Uber Eats,Uber,"Houston, TX",NaN,linkedin,No
7368,8298861,Deliver with Uber Eats,Uber,"Wimberley, TX",NaN,linkedin,No
2463,4bb08d3d06020768,Supv-Operations (S),FedEx Freight,"Edwardsville, KS 66111",NaN,indeed,Yes
14322,6166867,Deliver with Uber Eats,Uber,"Dearborn Heights, MI",NaN,linkedin,No


In [3]:
df.shape

(16826, 7)

In [4]:
df[df['Job Title Present']=='Yes']

,doc_id,doc_title,doc_company,doc_location,parsed_jt,source,Job Title Present
0,44f39ed2da88489f,Supv-Operations,FedEx Freight,"Monmouth Junction, NJ 08852",NaN,indeed,Yes
1,2e0698bce0078853,Supv-Operations,FedEx Freight,"Nazareth, PA 18064",NaN,indeed,Yes
2,5d21026101f668ce,Supv-Operations,FedEx Freight,"Fremont, IN 46737",NaN,indeed,Yes
3,01730086fb47951f,Supv-Operations,FedEx Freight,"Buffalo, NY 14217",NaN,indeed,Yes
4,2a5afa0cf0623df3,Supv-Operations,FedEx Freight,"Bismarck, ND 58504",NaN,indeed,Yes
...,...,...,...,...,...,...,...
16820,9743824,Lead- 5:00am –1:30pm,FedEx,"Memphis, TN",NaN,linkedin,Yes
16822,111747120,Summer 2023 Internship/Co-Op,Tennessee Valley Authority,"Chattanooga, TN",NaN,linkedin,Yes
16823,134756221,Driver Port Charlotte,Hertz,"Port Charlotte, FL",NaN,linkedin,Yes
16824,113301112,Northeast Region Intern,CALSTART,"California, United States",NaN,linkedin,Yes


In [5]:
from openai import OpenAI
openai_client = OpenAI()

In [6]:
from pydantic import BaseModel
class JobTitleExtraction(BaseModel):
    job_title: list[str]
    experience_level: list[str]
    specialization: list[str]
    location: list[str]

In [7]:
def llm_ner_job_title(job_title, client):
    if not client:
        print('No client for LLM usage')
        return None
    
    def prepare_job_title(job_title):
        if not isinstance(job_title, str):
            job_title = str(job_title)
        delimiters = ['-', '/', '_', ',', '.']
        for delimiter in delimiters:
            job_title = job_title.replace(delimiter, f' {delimiter} ')  # Separate special characters to facilitate labelling
        return job_title


    # Separe the special characters from the alphabetic characters
    job_title_prepared = prepare_job_title(job_title)
    print(job_title_prepared)


    #chat_completion = client.chat.completions.create(
    chat_completion = client.beta.chat.completions.parse(
        model='gpt-4o-mini',
        temperature=0.0,
        seed=42,
        messages=[
            {
            "role": "system",
            "content": """
            You are an expert in Renewable Energies with specialized skills in performing Named Entity Recognition (NER) on job titles. Your task is to extract specific entities from the provided job titles, without altering the original text.

            The entities to extract are:
            - 'job_title': Words directly related to the core occupation (example: 'Wind Turbine Technician')
            - 'experience_level': Words indicating the worker's seniority or experience level (example: 'Senior')
            - 'location': Any words that refer to geographical locations (example: 'California')
            - 'specialization': Additional words that give specific context or specialization to the job title (example: 'offshore wind farms')

            Guidelines:
            - Your output must be a JSON object, where each key is an entity, and the values are lists of the words forming each entity.
            - Do not modify the original job title. Only include words present in the input.
            - Avoid including salary, timesheet, or unrelated information.
            - Be careful to avoid problematic characters in the output.

            Examples:
            - For the job title 'Financial Analyst II, emergent markets', return: {"job_title": ["Financial", "Analyst"], "experience_level": ["II"], "specialization": ["emergent", "markets"]}
            - For 'RS Avionics Electrical Technician Lead', return: {"job_title": ["Electrical", "Technician"], "experience_level": ["Lead"], "specialization": ["RS", "Avionics"]}
            - For 'Principal Renewables Energy Land Planner (Mid-Senior Level)', return: {"job_title": ["Land", "Planner"], "experience_level": ["Principal"], "specialization": ["Renewables", "Energy"]}

            IMPORTANT:
            - Perform NER only on the job title itself.
            - Do not include any explanations or additional text in the output—just the extracted entities in JSON format.
            """
            },
            {
                "role": "user",
                "content": f"Recognize the specified named entities on this job title: {job_title_prepared}"
            }
        ],
        response_format=JobTitleExtraction,


    )

    response_content = chat_completion.choices[0].message.content
    response_json = eval(response_content)

    return response_json

In [8]:
test  = 'Supv,Customer Experience-Billing'
response = llm_ner_job_title(job_title=test, client=openai_client)
response

Supv , Customer Experience - Billing


{'job_title': ['Customer', 'Experience'],
 'experience_level': ['Supv'],
 'specialization': ['Billing'],
 'location': []}

In [9]:
def generate_iob_tags(job_title, ner_output):
    """
    Generate IOB tags for the given job title based on NER output.

    :param job_title: str, the raw job title string
    :param ner_output: dict, the NER output with keys 'job_title', 'experience_level', 'specialization'
    :return: list, IOB tags corresponding to the job title words
    """

    def prepare_job_title(job_title):
        if not isinstance(job_title, str):
            job_title = str(job_title)
        delimiters = ['-', '/', '_', ',', '.']
        for delimiter in delimiters:
            job_title = job_title.replace(delimiter, f' {delimiter} ')  # Separate special characters to facilitate labelling
        return job_title
    
    # Separe the special characters from the alphabetic characters
    job_title_prepared = prepare_job_title(job_title)
    
    # Split the job title into words
    job_title_tokens = job_title_prepared.split()

    # Create a list to store IOB tags
    iob_tags = ['O'] * len(job_title_tokens)

    # Helper function to mark the IOB tags
    def mark_iob(tags, tokens, label):
        for idx, token in enumerate(tokens):
            token_index = job_title_tokens.index(token)
            if idx == 0:
                tags[token_index] = f'B-{label}'  # Beginning tag
            else:
                tags[token_index] = f'I-{label}'  # Inside tag

    # Process each entity and assign corresponding IOB tags
    if 'experience_level' in ner_output:
        mark_iob(iob_tags, ner_output['experience_level'], 'exp')
    
    if 'job_title' in ner_output:
        mark_iob(iob_tags, ner_output['job_title'], 'job')
    
    if 'specialization' in ner_output:
        mark_iob(iob_tags, ner_output['specialization'], 'spe')
    
    if 'location' in ner_output:
        mark_iob(iob_tags, ner_output['location'], 'loc')

    return (job_title_prepared,iob_tags)

In [10]:
# Example usage
job_title = "Supv, Customer Experience-Billing"


ner_output = {
    "job_title": ["Customer", "Experience"],
    "experience_level": ["Supv"],
    "specialization": ["Billing"]
}

training_tuple = generate_iob_tags(job_title, ner_output)
training_tuple

('Supv ,  Customer Experience - Billing',
 ['B-exp', 'O', 'B-job', 'I-job', 'O', 'B-spe'])

In [11]:
training_dataset = pd.DataFrame(columns=['job_titl_splitted','iob_tags'],data=[training_tuple])
training_dataset

,job_titl_splitted,iob_tags
0,"Supv , Customer Experience - Billing","[B-exp, O, B-job, I-job, O, B-spe]"
